# Generate Test Questions

In [1]:
from training.training_data_generator import TrainingDataGenerator

# Initialize training data generator
generator = TrainingDataGenerator(
    question_bank_path='training/question_bank.md', 
    trees_csv_path='data/trees_df.csv', 
    patches_csv_path='data/patches_df.csv'
)

# Generate training data
df = generator.generate_dataset()
print(f"Generated {len(df)} training examples")
print(f"\n{df['intent'].value_counts()}")

# Extract test questions
test_questions = df['text']

Generated 434 training examples

intent
level_requirements      104
tree_recommendations     91
payment                  78
growth_time              65
transportation           60
quest_requirements       36
Name: count, dtype: int64


# Test Intent Classifier

In [2]:
from source.intent_classifier import IntentClassifier

# Initialize intent classifier
classifier = IntentClassifier()

# Predict intent for each question
for question in test_questions:
    intent, confidence = classifier.predict(question)
    print(f'Question: {question}')
    print(f'Intent: {intent}')
    print(f'Confidence: {confidence}')
    print('')

Question: What item does the farmer need for orange trees?
Intent: payment
Confidence: 0.7824128354300635

Question: What tree should I plant at level 30?
Intent: tree_recommendations
Confidence: 0.8860529942418641

Question: What level do I need to plant pineapple?
Intent: level_requirements
Confidence: 0.8413021644728323

Question: What do I pay the farmer to protect banana?
Intent: payment
Confidence: 0.8254469521244336

Question: When will orange trees be ready?
Intent: growth_time
Confidence: 0.7327782926877697

Question: How long until magic is fully grown?
Intent: growth_time
Confidence: 0.7637564555806287

Question: What payment is needed for willow trees?
Intent: payment
Confidence: 0.6961162895746562

Question: What tree is best at level 27?
Intent: tree_recommendations
Confidence: 0.815467588046192

Question: How long until curry is fully grown?
Intent: growth_time
Confidence: 0.8263890813823759

Question: What is the fastest way to the taverley patch?
Intent: transportation

# Test Answer Retriever

In [3]:
from source.intent_classifier import IntentClassifier
from source.answer_retriever import AnswerRetriever

# Initialize intent classifier
classifier = IntentClassifier()

# Initialize answer retriever
retriever = AnswerRetriever()

# Predict intent for each question
for question in test_questions:
    intent, confidence = classifier.predict(question)
    lookup_value, answer_value, recommended = retriever.get_answer(question, intent)
    print(f'Question: {question}')
    print(f'Lookup: {lookup_value}')
    print(f'Intent: {intent}')
    print(f'Answer: {answer_value}')
    print('')

Question: What item does the farmer need for orange trees?
Lookup: orange
Intent: payment
Answer: 3 baskets of strawberries

Question: What tree should I plant at level 30?
Lookup: 30
Intent: tree_recommendations
Answer: willow

Question: What level do I need to plant pineapple?
Lookup: pineapple
Intent: level_requirements
Answer: 51

Question: What do I pay the farmer to protect banana?
Lookup: banana
Intent: payment
Answer: 4 baskets of apples

Question: When will orange trees be ready?
Lookup: orange
Intent: growth_time
Answer: 16 hours

Question: How long until magic is fully grown?
Lookup: magic
Intent: growth_time
Answer: 8 hours

Question: What payment is needed for willow trees?
Lookup: willow
Intent: payment
Answer: 1 basket of apples

Question: What tree is best at level 27?
Lookup: 27
Intent: tree_recommendations
Answer: apple

Question: How long until curry is fully grown?
Lookup: curry
Intent: growth_time
Answer: 16 hours

Question: What is the fastest way to the taverley 

# Test Response Generator

In [ ]:
from source.intent_classifier import IntentClassifier
from source.answer_retriever import AnswerRetriever
from source.response_generator import ResponseGenerator

# Initialize intent classifier
classifier = IntentClassifier()

# Initialize answer retriever
retriever = AnswerRetriever()

# Initialize response generator
generator = ResponseGenerator()

# Predict intent for each question
for question in test_questions:
    intent, confidence = classifier.predict(question)
    lookup_value, answer_value, recommended = retriever.get_answer(question, intent)
    response = generator.generate_response(intent, lookup_value, answer_value, recommended)
    print(f'Question: {question}')
    print(f'Lookup: {lookup_value}')
    print(f'Intent: {intent}')
    print(f'Answer: {answer_value}')
    print(f'Recommended: {recommended}')
    print(f'Response: {response}')
    print('')

Question: What item does the farmer need for orange trees?
Lookup: orange
Intent: payment
Answer: 3 baskets of strawberries
Response: Orange trees can be protected by giving the farmer 3 baskets of strawberries.

Question: What tree should I plant at level 30?
Lookup: 30
Intent: tree_recommendations
Answer: willow
Response: At level 30 Farming, you should plant Willow trees.

Question: What level do I need to plant pineapple?
Lookup: pineapple
Intent: level_requirements
Answer: 51
Response: Pineapple trees require level 51 Farming to plant.

Question: What do I pay the farmer to protect banana?
Lookup: banana
Intent: payment
Answer: 4 baskets of apples
Response: Banana trees can be protected by giving the farmer 4 baskets of apples.

Question: When will orange trees be ready?
Lookup: orange
Intent: growth_time
Answer: 16 hours
Response: Orange trees will be fully grown after 16 hours.

Question: How long until magic is fully grown?
Lookup: magic
Intent: growth_time
Answer: 8 hours
Resp